In [1]:
import pylids

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
# Check for available model weights
pylids.available_models()

Available models

['eyelids_v1', 'eyelids_pupils_v1', 'eyelids_pupils_v2']


- The best eyelids only model is eyelids_v1
- For pupils and gaze use eyelids_pupils_v2

In [3]:
#This is the main function you want to use to estimate pupil and eyelids
#checkout the docstring for how to use
pylids.analyze_video?

In [4]:
#running these on included demo eye videos
ls

eye0.mp4  eye1.mp4  eye_annotation.mp4  pylids_demo.ipynb


In [ ]:
#Demo run
pylids_out = pylids.analyze_video(eye_vid='eye1.mp4', model_name= 'eyelids_pupils_v2', save_vid=True)